#Chisholm / Prochlorococcus project
##Use Python and its various libraries to link the KEGG compound information with pathways
###KLongnecker, 4/17/2017

In [435]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [436]:
import pandas as pd
import numpy as np
import re
import os
import glob
import math
from imp import reload

import palettable as pal
import matplotlib.pyplot as plt
import matplotlib as mpl 
mpl.rcParams['pdf.fonttype'] = 42

from Bio import SeqIO
from Bio.KEGG.REST import *
from Bio.KEGG.KGML import KGML_parser
from Bio.Graphics.KGML_vis import KGMLCanvas
from IPython.display import Image, HTML

#used this to step into the function and debug it, also need line with Tracer()() 
from IPython.core.debugger import Tracer 
#os._exit(1) #this should get me out of the debugging...

%matplotlib inline

In [437]:
mtabFile = 'ChisholmPro_cellSpecific_KEGGexport.2017.04.18.csv' #first column is KEGG C number
mtabData=pd.read_csv(mtabFile, index_col='KEGG')
mtabData.head()

,Plimited_extracellular_s9301ax_50,Plimited_filter_s9301ax_50,replete_extracellular_s9301ax_10,replete_extracellular_s9301ax_50,replete_extracellular_s0801ax_10,replete_extracellular_s9313ax_5,replete_extracellular_s9313ax_10,replete_filter_s9301ax_10,replete_filter_s9301ax_50,replete_filter_s0801ax_10,replete_filter_s9313ax_5,replete_filter_s9313ax_10
KEGG,,,,,,,,,,,,
C00196,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
C19675,1.502503e-09,0.0,7.559786e-09,0.000000e+00,8.830372e-09,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
C08276,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
C00141,1.501737e-07,0.0,1.269247e-07,1.719122e-07,3.699587e-09,1.385979e-06,1.185440e-06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
C00671,0.000000e+00,0.0,5.760611e-08,1.074612e-07,0.000000e+00,3.044114e-07,3.038489e-07,1.499082e-09,1.232101e-09,2.171711e-09,1.414847e-10,2.189543e-09


In [438]:
if False: 
    #this will be useful later...but is apples and oranges right now...
    ancillaryFile = 'ChisholmPro_ancillaryData.2017.04.18.csv'
    ancillaryData = pd.read_csv(ancillaryFile, index_col = None)

In [439]:
#allPathways = kegg_list("pathway").read() #all organisms
allPathways = kegg_list("pathway","pmg").read() #pmg i 9301, HLII
len(allPathways.split('\n'))

99

In [440]:
trimPath = []
current_section = None
for line in allPathways.rstrip().split("\n"):
    tp = line[5:13]
    trimPath.append(tp)

In [441]:
#set up a function to get the list of compounds for a given pathway (must be defined as ko00140 NOT map00140)
def getCfrom_ko(ko_id):
    pathway_file = kegg_get(ko_id).read()  # query and read the pathway
    compound_list = []

    current_section = None
    for line in pathway_file.rstrip().split("\n"):
        section = line[:12].strip()  # section names are within 12 columns
        if not section == "":
            current_section = section
        if current_section == "COMPOUND":
            compound_identifiers = line[12:].split("; ")
            t = compound_identifiers[0]
            compound_id = t[0:6]

            if not compound_id in compound_list:
                compound_list.append(compound_id)
    return compound_list

In [442]:
#set up a function to get the list of K orthologues for a given pathway (must be defined as ko00140 NOT map00140)
#for this project: can use the three digit code for the different Prochlorococcus strains
def getKfrom_ko(ko_id):
    pathway_file = kegg_get(ko_id).read()  # query and read the pathway
    K_list = []

    current_section = None
    for line in pathway_file.rstrip().split("\n"):
        section = line[:12].strip()  # section names are within 12 columns
        if not section == "":
            current_section = section
        if current_section == "ORTHOLOGY":
            K_identifiers = line[12:].split("; ")
            t = K_identifiers[0]
            K_id = t[0:6]

            if not K_id in K_list:
                K_list.append(K_id)
    return K_list

In [443]:
# A bit of code that will help us display the PDF output
def PDF(filename):
    return HTML('<iframe src=%s width=700 height=350></iframe>' % filename)

In [444]:
import fxn_plotPathway
##if I make a change, have to reload the function:
# reload(fxn_plotPathway)

In [445]:
mtabData.index

Index(['C00196', 'C19675', 'C08276', 'C00141', 'C00671', 'C00568', 'C00156',
       'C00233', 'C00170', '0', 'C00352', 'C04022', 'C00334', 'C01279',
       'C00004', 'C00134', '0', 'C00147', 'C00212', 'C00020', 'C00026',
       'C00062', 'C00049', 'C00719', 'C00120', 'C07481', 'C01674', '0',
       'C00114', 'C03557', 'C00158', 'C00327', 'C08230', 'C00380', 'C01909',
       'C00111', 'C06231', 'C00504', 'C06454', 'C00122', 'C00092', 'C00025',
       'C00064', 'C01705', 'C00242', 'C00387', '0', 'C00954', 'C00294',
       'C00130', 'C00407', 'C00097', 'C00051', 'C00328', 'C00123', '0',
       'C00149', 'C00073', 'C06470', 'C00140', 'C00624', 'C02713', 'C00077',
       'C00295', 'C00864', 'C00079', 'C00074', 'C05786', 'C00148', 'C00314',
       'C00255', '0', 'C00021', 'C00019', 'C00213', 'C00065', 'C00093',
       'C05122', '0', 'C00042', 'C10833', 'C00245', 'C00378', 'C01081',
       'C00188', 'C00214', 'C00398', 'C00078', 'C00106', 'C00105', 'C00183',
       'C00385', 'C01762'],
      

In [446]:
#need to delete the zeros for now...
mtabPruned = mtabData.drop(['0'])

In [447]:
mtabPruned.index

Index(['C00196', 'C19675', 'C08276', 'C00141', 'C00671', 'C00568', 'C00156',
       'C00233', 'C00170', 'C00352', 'C04022', 'C00334', 'C01279', 'C00004',
       'C00134', 'C00147', 'C00212', 'C00020', 'C00026', 'C00062', 'C00049',
       'C00719', 'C00120', 'C07481', 'C01674', 'C00114', 'C03557', 'C00158',
       'C00327', 'C08230', 'C00380', 'C01909', 'C00111', 'C06231', 'C00504',
       'C06454', 'C00122', 'C00092', 'C00025', 'C00064', 'C01705', 'C00242',
       'C00387', 'C00954', 'C00294', 'C00130', 'C00407', 'C00097', 'C00051',
       'C00328', 'C00123', 'C00149', 'C00073', 'C06470', 'C00140', 'C00624',
       'C02713', 'C00077', 'C00295', 'C00864', 'C00079', 'C00074', 'C05786',
       'C00148', 'C00314', 'C00255', 'C00021', 'C00019', 'C00213', 'C00065',
       'C00093', 'C05122', 'C00042', 'C10833', 'C00245', 'C00378', 'C01081',
       'C00188', 'C00214', 'C00398', 'C00078', 'C00106', 'C00105', 'C00183',
       'C00385', 'C01762'],
      dtype='object', name='KEGG')

In [448]:
if False:
    #do all pathways
    for item in trimPath:
        fxn_plotPathway.gatherDetails(item,useFolder,useCO,CO_values)

In [449]:
##next up...count mtabs by pathway (and/or metabolism). 
#This is where the BRITE pathway comes into play

In [450]:
#organize pathways into the groups defined in the BRITE file
def ReadBRITEfile(briteFile):
    forBrite = pd.DataFrame(columns = ['map','A','B','C','wholeThing'])
    # set up the expressions to match each level in the BRITE hierarchy
    
    textA = re.compile(r'(^A<b>)(.+)(</b>)\s*(.*)$')
    textB = re.compile(r'(^B)\s*(.*)$')
    textC = re.compile(r'(\d+)\s*(.*)$')
    #this relies on the fact that the rows are in order: A, with B subheadings, then C subheadings
    setA = []
    idxA = []

    setB = []
    setC = []

    with open(briteFile) as f:
        for idx,line in enumerate(f):
            if line[0] is not '#': #skip over the comments
                mA = textA.search(line) 
                mB = textB.search(line) 
                mC = textC.search(line) 
                if mA:
                    setA = mA.group(2)
                    #house cleaning (probably c)
                    idxA = idx
                    forBrite.loc[idx,'A'] = setA
                    forBrite.loc[idx,'wholeThing'] = line #using this as a double check for now
                    #forBrite.loc[idx,'map'] = mC.group(1)
                elif mB:
                    setB = mB.group(2)
                    forBrite.loc[idx,'A'] = setA
                    forBrite.loc[idx,'B'] = setB
                    forBrite.loc[idx,'wholeThing'] = line
                    #forBrite.loc[idx,'map'] = mC.group(1)
                elif mC:
                    #Tracer()()
                    setC = mC.group(2)
                    forBrite.loc[idx,'A'] = setA
                    forBrite.loc[idx,'B'] = setB
                    forBrite.loc[idx,'C'] = setC
                    forBrite.loc[idx,'wholeThing'] = line
                    forBrite.loc[idx,'map'] = mC.group(1)

    return forBrite

In [451]:
#go read in the file from KEGG
D = glob.glob('br08901.keg') #from http://www.genome.jp/kegg-bin/get_htext?br08901.keg; 4/18/2017
allBRITE=[]
for idx,nof in enumerate(D):
    allBRITE = ReadBRITEfile(nof)

In [452]:
allBRITE.head()

,map,A,B,C,wholeThing
8,NaN,Metabolism,NaN,NaN,A<b>Metabolism</b>\n
9,NaN,Metabolism,Global and overview maps,NaN,B Global and overview maps\n
10,01100,Metabolism,Global and overview maps,Metabolic pathways,C 01100 Metabolic pathways\n
11,01110,Metabolism,Global and overview maps,Biosynthesis of secondary metabolites,C 01110 Biosynthesis of secondary metaboli...
12,01120,Metabolism,Global and overview maps,Microbial metabolism in diverse environments,C 01120 Microbial metabolism in diverse en...


In [453]:
prunedBRITE = allBRITE.dropna()

In [454]:
prunedBRITE.head()

,map,A,B,C,wholeThing
10,01100,Metabolism,Global and overview maps,Metabolic pathways,C 01100 Metabolic pathways\n
11,01110,Metabolism,Global and overview maps,Biosynthesis of secondary metabolites,C 01110 Biosynthesis of secondary metaboli...
12,01120,Metabolism,Global and overview maps,Microbial metabolism in diverse environments,C 01120 Microbial metabolism in diverse en...
13,01130,Metabolism,Global and overview maps,Biosynthesis of antibiotics,C 01130 Biosynthesis of antibiotics\n
14,01200,Metabolism,Global and overview maps,Carbon metabolism,C 01200 Carbon metabolism\n


In [455]:
#probably a way to do this in one step:
findM = prunedBRITE.loc[(prunedBRITE['A'] == 'Metabolism')]
findU = findM.loc[:,'B']

In [456]:
#really not that many pathway groups
findU.unique()

array(['Global and overview maps', 'Carbohydrate metabolism',
       'Energy metabolism', 'Lipid metabolism', 'Nucleotide metabolism',
       'Amino acid metabolism', 'Metabolism of other amino acids',
       'Glycan biosynthesis and metabolism',
       'Metabolism of cofactors and vitamins',
       'Metabolism of terpenoids and polyketides',
       'Biosynthesis of other secondary metabolites',
       'Xenobiotics biodegradation and metabolism',
       'Chemical structure transformation maps'], dtype=object)

In [521]:
unMetabolismGroups = findU.unique()
#skip over the global and overview maps
unMetabolismGroups = np.delete(unMetabolismGroups,(0),axis=0)

In [522]:
unMetabolismGroups

array(['Carbohydrate metabolism', 'Energy metabolism', 'Lipid metabolism',
       'Nucleotide metabolism', 'Amino acid metabolism',
       'Metabolism of other amino acids',
       'Glycan biosynthesis and metabolism',
       'Metabolism of cofactors and vitamins',
       'Metabolism of terpenoids and polyketides',
       'Biosynthesis of other secondary metabolites',
       'Xenobiotics biodegradation and metabolism',
       'Chemical structure transformation maps'], dtype=object)

In [523]:
#now that I have the colors all set...let's think about what we actually want to plot...
list(mtabPruned)

['Plimited_extracellular_s9301ax_50',
 'Plimited_filter_s9301ax_50',
 'replete_extracellular_s9301ax_10',
 'replete_extracellular_s9301ax_50',
 'replete_extracellular_s0801ax_10',
 'replete_extracellular_s9313ax_5',
 'replete_extracellular_s9313ax_10',
 'replete_filter_s9301ax_10',
 'replete_filter_s9301ax_50',
 'replete_filter_s0801ax_10',
 'replete_filter_s9313ax_5',
 'replete_filter_s9313ax_10']

In [618]:
#run for one strain (options are 'pmg','pmt','prm')
oneStrain = 'pmg'
useCO = mtabPruned.index
CO_values = mtabPruned.loc[:,'Plimited_filter_s9301ax_50']/mtabPruned.loc[:,'replete_filter_s9301ax_50']  

for item in unMetabolismGroups:
    oneGroup = item
    shortList = prunedBRITE.loc[(prunedBRITE['B']==oneGroup)] 
    onePath = shortList.loc[:,'map']
    onePath_ann = []
    for item in onePath:
        onePath_ann.append(oneStrain + item)
    #do some pathways, divide by group, but...can have pathways not annotated for this Pro strain
    for item in onePath_ann:    
        try:
        #print(item)
            fxn_plotPathway.gatherDetails(item,oneGroup,useCO,CO_values)
        except:
            print('problem with this pathway ' + item)

problem with this pathway pmg00351
problem with this pathway pmg01010
problem with this pathway pmg01060
problem with this pathway pmg01061
problem with this pathway pmg01062
problem with this pathway pmg01063
problem with this pathway pmg01064
problem with this pathway pmg01065
problem with this pathway pmg01066
problem with this pathway pmg01070


In [614]:
if True:
    #do one pathway, best for testing
    useFolder = 'testingFolder'
    usePathway = 'pmg00908'
    useCO = mtabPruned.index
    #drat...ratios will cause problems bc inf (changed function: 0/inf will be top and bottom of color scale)
    CO_values = mtabPruned.loc[:,'Plimited_filter_s9301ax_50']/mtabPruned.loc[:,'replete_filter_s9301ax_50']  
    #CO_values = mtabPruned.loc[:,'Plimited_filter_s9301ax_50']
    fxn_plotPathway.gatherDetails(usePathway,useFolder,useCO,CO_values)

In [ ]:
#leave debris below here for now

In [617]:
#only need to reload if I change the *py file
reload(fxn_plotPathway)

<module 'fxn_plotPathway' from 'C:\\Users\\krista\\Documents\\GitHub\\Pro_mtabs\\fxn_plotPathway.py'>